In [2]:
import sys
import random
import time

from settings import Settings
from script import Script
from events import Events

class Game:
    
    def __init__(self):
        
        self.settings = Settings()
        self.script = Script()
        self.events = Events(self)

        self.autogame = 0
        self.input = 0
        #自动间隔时间
        self.sleeptime = 2 
           
    def run_game(self):
        """游戏主循环"""
        print("|||———————————————————-|||")
        print(self.script.title)
        print("|||__________SIEGE--OF--HENGSHAI__________|||")
        #开始界面
        self.prologue()
        while self.settings.game:
            #数值
            self.update()
            #当前主事件
            self.total_game()
            #检查是否解封
            self._check_end()
            self._check_states()
            self.debug()
            #自动游戏后加入时间间隔
            #检查输入框
            self._check_input()
            if self.autogame == 1:
                time.sleep(self.sleeptime)
        if self.settings.ending:
            self.ending()
        input("\n按任意键退出游戏。")
    
    def update(self):

        self.settings.day += 1
        self.settings.luck = random.randint(1,10)
        
        #鸡蛋
        if self.settings.beggining:
            self.settings.eggs -=2
            if self.settings.eggs <=0:
                self.settings.mood -= 1
                self.settings.eggs = 0
        #根据日期切换城市状态。
        if self.settings.day == 5:
            self.settings.mode = 1
            
        elif self.settings.day ==20:
            self.settings.mode = 2
                                    
    def total_game(self):
        
        """决定选择哪个主事件"""
        if self.settings.arc_game:
            #方舱
            self.arc_game()
        else:
            #主事件
            self.main_game()
        
    def main_game(self):
        
        """主要故事线"""
        print(f"\n【今天是封城的第{self.settings.day}天】")
        self.events.operations_events()
        self.events.important_events()
        self.events.NA_events()
        self.events.random_events()
        self.events.cyber_events()
        self.events.sleep_events()
        #self._check_hungry()
        #阳性检定
        self._check_positive()
    
    def arc_game(self):
        
        """方舱故事线"""
        self.settings.arc_day +=1
        print(f"\n【今天是封城的第{self.settings.day}天】")
        print(f"【今天是来到方舱后的第{self.settings.arc_day}天】")        
        self.events.arc_events()
        #出舱判定
        if self.settings.arc_day ==self.settings.positive_days :
            self.settings.beggining = True
            self.settings.arc_game = False
            self.settings.immune = 10
            print(self.script.negative1)
                
            
    def _check_begin(self):
        """游戏开始前数值初始化"""
        str = input("是否开始游戏？(y/n)")
        if str.upper() =='Y':
            self.settings.prologue = False
            self.settings.beggining = True
            self.settings.game = True
            self.settings.immune = self.settings.strength
            self.settings.eggs = self.settings.wealth*3+10
            sys.stdout = Logger("my_diary.txt")
            print("\n当我年轻的时候，我记得任何事情，无论它是否发生过。——马克.吐温")
            
        elif str.upper() == 'N':
            self.prologue()
        else:
            print("请输入正确的选择(y/n)")
            self._check_begin()
            
    def _check_end(self):
        """检查封锁日期"""
        if self.settings.day == self.settings.days:
            self.clear_all()
            self.settings.ending =True
            
    def _check_states(self):
        
        while self.settings._check_eggs and self.settings.beggining:
            
            if self.settings.eggs <=0:
                print("\n你消耗完了所有的鸡蛋，失去了重要的蛋白质补充。\n你的免疫力下降了，你的心情也开始变得更糟。")
                self.settings.immune -=1
                self.settings._check_eggs = False
            break
            
        while self.settings._check_mood:
            if self.settings.mood <=0:
                print("\n高压下的生活让你的心情跌倒了极点。\n你的免疫力下降了。")
                self.settings.immune -=1
                self.settings._check_mood = False
            break
        if self.settings.mood <=0:
            self.settings.mood = 0
            
    def _check_input(self):
        """输入检查"""
        while self.autogame == 0:
            self.input = input("\n...任意键继续，按a自动游戏，按e退出游戏...")
            if self.input.upper() == 'A':
                self.autogame =1
            elif self.input.upper() =='E':
                self.clear_all()
                print("\n游戏终止，感谢您进行游戏。")                    

            break

    def _check_positive(self):
        """阳性检测"""
        if self.settings.immune <=0 and self.settings.day+7 <self.settings.days :
            print(self.script.positive)
            self.settings.end_arc = True
            if self.settings.luck >8:
                #会展中心。
                print(self.script.positive_luck)
            else:
                #无家可归。
                print(self.script.positive_fuck)
                input("")
                print(self.script.positive_fuck2)
                input("")
                print(self.script.positive_fuck3)
                self.settings.mood -=20
                self.settings.end_travel =True
                
            self.settings.beggining = False
            self.settings.arc_game = True
            
    def clear_all(self):
        """清理所有主循环的flag,给结尾制造条件"""
        self.settings.beggining = False
        self.settings.arc_game = False
        self.settings.game = False
    
    def _get_information(self):
        """自定义人物部分"""
        print("\n欢迎游玩本游戏，请定制你的人物，\n你的人物属性会影响游戏进程。\n【四项属性上限为20点】")
        try:
            self.settings.wealth = abs(int(input("请选择你的壕力")))
            self.settings.strength = abs(int(input("请选择你的体格")))
            self.settings.Ingenuity = abs(int(input("请选择你的才智")))
            self.settings.Cyber = 20 - self.settings.wealth -self.settings.strength - self.settings.Ingenuity 
        except ValueError:
            print("\n识别失败，请输入数字")
            self._get_information()
                
    def prologue(self):
        '''开始界面'''
        while self.settings.prologue:
            self._get_information()

            if self.settings.Cyber < 0 :
                print("\n超出最大分配点数！")
                continue

            print(f"\n您的壕力为{self.settings.wealth}")
            print(f"您的体格为{self.settings.strength}")
            print(f"您的才智为{self.settings.Ingenuity}")
            print(f"您的网龄为{self.settings.Cyber}\n")

            self._check_begin()
            
    def ending(self):
        """尾声和分支结局"""       
        print(self.script.end_1)
        input('')
        #print(self.settings.reputation)
        if self.settings.reputation >= 3:
            print(self.script.end_reputation)
            input('')
        
        #方舱分支
        if self.settings.end_arc == True:
            print(self.script.end_arc)
            input('')
            if self.settings.end_travel ==True:
                print(self.script.end_arc_travel)
                input('')
        else:
            print(self.script.end_arc_none)
            input('')
            
        #张麻子
        if self.settings.friendship_zmz:
            print(self.script.end_zmz)
            input('')
        
        if self.settings.mood>60:
            print(self.script.end_mood)
        else:
            print(self.script.end)
            
    def debug(self):
        #print(f"\n免疫值：{self.settings.immune},鸡蛋数目：{self.settings.eggs},心情：{self.settings.mood}\n运气：{self.settings.luck},声望：{self.settings.reputation}")
        print(f"\n健康：{self.settings.immune},心情：{self.settings.mood},声望：{self.settings.reputation}，城市紧张度：{self.settings.mode+1}，垃圾桶：{self.settings.default}")


#储存任务日记
class Logger(object):

    def __init__ (self, fileN="Default.1og"):
        self.terminal = sys.stdout
        self.log = open(fileN,"w",encoding='utf-8')

    def write(self, message):
        self.terminal.write(message)
        self.log.write(message)

    def flush(self):
        pass        

#
if __name__ == '__main__':
       
    siege = Game()
    siege.run_game()
   

|||———————————————————-|||
⠄⠄⣀⣤⣤⣤⣄⡀⠄⢀⣠⣤⣤⣄⡀⠄⢀⣠⣤⣤⣄⡀⢀⣠⣤⣤⣄⡀⠄⠄
⢀⣾⣿⡿⠿⠿⣿⣿⣦⢻⡿⠟⠿⣿⣿⣦⡹⣿⠿⠿⣿⣿⣦⠹⠿⠿⣿⣿⣧⡀
⠸⠿⠿⠄⠄⠄⢸⣿⣿⠄⠄⠄⠄⠈⢿⣿⣧⠄⠄⠄⠈⣿⣿⡇⠄⠄⠈⣿⣿⡇
⠄ ⠄⠄⠄⣠⣾⣿⡟⠄⠄⠄⠄⠄⢸⣿⣿.⠄⠄⢀⣼⣿⡿⠁⠄⢀⣴⣿⡿
⠄⠄⢀⣠⣾⣿⠟⢋⣴⠄⠄⠄⠄⠄⢸⡿⠋⣠⣾⣿⠟⠋⠄⣠⣶⣿⠿⠋⠄⠄
⠄⣰⣿⡿⠋⠄⠈⣿⣿⣦⠄⠄⠄⢀⡾⣡⣾⣿⠋⠁⠄⢠⣾⣿⠟⠁⠄⠄⠄⠄
⢰⣿⣿⣿⣶⣶⣶⣌⠻⣿⣿⣶⣶⣿⢱⣿⣿⣿⣷⣶⠂⣿⣿⣿⣷⣶⣶⣶⣶⡆
⠈⠉⠉⠉⠉⠉⠉⠉⠁⠈⠙⠛⠛⠉⠈⠉⠉⠉⠉⠉⠄⠉⠉⠉⠉⠉⠉⠉⠉⠁⠀
|||__________SIEGE--OF--HENGSHAI__________|||

欢迎游玩本游戏，请定制你的人物，
你的人物属性会影响游戏进程。
【四项属性上限为20点】
请选择你的壕力4
请选择你的体格5
请选择你的才智6

您的壕力为4
您的体格为5
您的才智为6
您的网龄为5

是否开始游戏？(y/n)y

当我年轻的时候，我记得任何事情，无论它是否发生过。——马克.吐温

【今天是封城的第1天】
上面要求打赢疫情防控大仗硬仗。

突然起来的封城令所有横沙人都措手不及，但你是一名死肥宅。
由于平时有囤积物资的意识，封城并没有对你造成太多的困扰。
你的同事并没有那么好运，他们当中有些人平时靠外卖度日，在封城前只准备了不到一周的干粮，他们开始对可能迎来的食品短缺感到不安。
这种焦虑很快蔓延到了整个公司，居家办公的效率自然无从谈起。

为了让自己的心态保持平和，你选择让自己与现实脱钩，再一次隐身于伟大的互联网。

但很快你发现，不同空间与不同时间的负面情绪都汇集于此，
互联网所展现出来的横沙比你自身感受到的要严峻得多。
无奈之下，你打开了Steam，这里有着上千款你买来从未玩过的游戏等待着你的幸临。

不管怎么说，你今晚的心情还算可以,
你决定多打一会游戏再睡觉。
健康：5,心情：60,声望：0，城市紧张度：1，垃圾桶：100

...任意键继续，按a自动游戏，按e退出游戏...

【今天是封城的第2天】
今天不需要做核酸。
居委将试剂盒分装好，送到了楼下大堂。
你被告知一天要测一次抗原,
最后要将抗原结果传到小程序上。

你在微博上刷到了一个视频,
视频里有一名白大褂用棉签在楼道内涂抹
视频的配文声称

【判定


...任意键继续，按a自动游戏，按e退出游戏...

【今天是封城的第26天】
【今天是来到方舱后的第4天】
今天的核酸检测点看起来和以往有些不太一样，
负责扫码的医护人员面前多了个大纸箱，
他在里面透过小洞来检测每一个人的二维码

【判定成功】：
你仔细一想，此举是为了减少扫码时太阳的反光，提高扫码效率。
简单但实用，这就是劳动人民的智慧。
健康：0,心情：60,声望：4，城市紧张度：3，垃圾桶：125

...任意键继续，按a自动游戏，按e退出游戏...

【今天是封城的第27天】
【今天是来到方舱后的第5天】
你在床上无聊地玩着手机，突然间远方的人群变得喧闹了起来。

【判定成功】：
多年网上冲浪的经验告诉你，来者正是王思聪。哟王校长也有今天。
健康：0,心情：60,声望：4，城市紧张度：3，垃圾桶：126

...任意键继续，按a自动游戏，按e退出游戏...

【今天是封城的第28天】
【今天是来到方舱后的第6天】
上厕所是隔离期间最为痛苦的一件事。
每一个隔间都犹如一个潘多拉魔盒，
你永远不知道打开一扇门后要面对什么。

【判定成功】：
你知道工作人员每天都会定时打扫厕所。
在经过长时间的观察后，你摸清了他们的工作时间。
自此，你每次都能享受到最干净的厕所。
健康：0,心情：60,声望：4，城市紧张度：3，垃圾桶：127

...任意键继续，按a自动游戏，按e退出游戏...

【今天是封城的第29天】
【今天是来到方舱后的第7天】
你在测核酸的时候，注意到不远处有人聚集在了一起。
只见几个蒙面大白将一块白布在地上铺展开。
上面按一定行列间隔摆满了各种饮料、零食还有方便面。
一群成年人围在一起，玩起了套圈圈。
方舱的娱乐活动，让你找回了儿时的感觉。

【判定成功】：
你跃跃欲试，上前尝试了几次。
当你最终回到自己的床位时，身边还多了一听肥宅快乐水。

你在过去24小时的两次核酸结果都呈阴性
因此你顺利地拿到了解离书。
方舱的专车将你们送到街道门口便匆匆离去。
虽然七天没有回家，但你并不急急着回小区。
即便你是一个肥宅，你依然享受伴着晚风在梧桐树下慢慢行走的时光。
横沙的街头总是熙熙攘攘，你从未见过这里的街道能如此宁静祥和。

从街道入口走到小区门口并没有花多少时间。
工作人员在确认了你的身份和解离书后，将你放进了小区。
谢天谢地，你并没有像网上流传的那样无

In [1]:
#提取文本
from script import Script
import xlsxwriter as xw
import plotly.express as px

data = Script()

mode = 0
quest = 0

ids = [] 
description = []
type1 = []
value = []
succeed = []
fail = []
succeed_type = []
succeed_value = []
fail_type = []
fail_value = []


for mode in range (0,3):
    for quest in range (0,len(data.random_quests[mode])):
        ids.append(str(data.random_quests[mode][quest]['id']))
        description.append(str(data.random_quests[mode][quest]['fields']['description']))
        type1.append(str(data.random_quests[mode][quest]['fields']['type']))
        value.append(str(data.random_quests[mode][quest]['fields']['value']))
        succeed.append(str(data.random_quests[mode][quest]['fields']['succeed']))
        fail.append(str(data.random_quests[mode][quest]['fields']['fail']))
        succeed_type.append(str(data.random_quests[mode][quest]['fields']['succeed_type']))
        succeed_value.append(str(data.random_quests[mode][quest]['fields']['succeed_value']))
        fail_type.append(str(data.random_quests[mode][quest]['fields']['fail_type']))
        fail_value.append(str(data.random_quests[mode][quest]['fields']['fail_value']))

print(description)
        
filename = 'test.xlsx'

workbook = xw.Workbook(filename)
worksheet1 = workbook.add_worksheet("self.random_quest")#创建子表
worksheet1.activate()#激活表
title = ['序号','任务叙述','判定类型','判定数值','成功检定','失败检定','成功结果','成功数值','失败结果','失败数值',]
worksheet1.write_row('A1',title)

i=2 
for j in range(0,len(description)):
    insertData = ids[j],description[j],type1[j],value[j],succeed[j],fail[j],succeed_type[j],succeed_value[j],fail_type[j],fail_value[j]
    row = 'A'+str(i)
    worksheet1.write_row(row,insertData)
    i+=1
workbook.close()


['任务简介', '测完核酸回程的途中，你听到邻居两夫妇在不停争吵。\n出于好奇，你决定凑上去看看发生了什么事。\n原来是丈夫团购一根菜没买，而是买了三箱可乐和数十条香烟。', '在排队测核酸时，你闲的无聊刷微博，\n看到横沙发布说今天的确诊只有300例。', '你注意到测核酸的队伍里有人人西装革履，有人衣着光鲜，有人油头粉面。\n好像测得不是核酸，测的是小资情调。', '你在微博上刷到了一个视频,\n视频里有一名白大褂用棉签在楼道内涂抹\n视频的配文声称', '任务简介', '在排队测核酸时，你闲的无聊刷微博，\n看到横沙发布说今天的确诊只有300例。', '你得知邻居家团购的鸡被偷走了，愤怒至极。', '你注意到头顶有异物飞过，原来是住顶楼的邻居开始翻出了吃灰多年的无人机，试图从观景池钓锦鲤。', '分发物资时，传来了一一阵喧闹声。\n一个老头执意认为物资应按照房子户数分发，并抢来了两盒鸡蛋。', '你注意到，业主群里开始分享水培蔬菜的经验，因为葱姜蒜在菜包里往往不会出现。', '小区团购的物资到了。', '你沉迷打魔兽，一直魔兽一直爽，封城的日子对肥宅而言真的太棒了。', '有人托关系搞到了购买肥宅快乐水的渠道，问你要不要一起参与。', '住在你隔壁的老人对线上支付并不熟悉，随即向你求助帮忙购买面粉', '你在睡午觉时，被阵阵哭声吵醒，\n原来是一名女子在四处敲门，哭着求退烧药。\n看的出来她病的很厉害。', '今天楼下测核酸的动静要远比平时热闹，\n你侧耳倾听，试图去了解发生了什么。', '你无意中找到了一个低价跑腿的渠道，购买了24个鸡蛋。', '今天楼下测核酸的动静要远比平时热闹，\n你侧耳倾听，试图去了解发生了什么。', '你在电视上看到，全球汽车芯片巨头IMESNO半导体突然宣布将关闭位于横沙的全球配送中心并将其迁往新加坡。', '隔壁的邻居粮食储备不够，找上门来提出用猫的使用权来换几枚鸡蛋。', '粉红色的天空下着玫瑰色的细雨。\n你像往常一样听从指示下楼测核酸。\n突然，前方传来了一阵喧哗声，周围的人解释说\n有家长让小孩跳《听我说谢谢你》，\n后面的人嫌浪费时间，就打了起来。', '今天的核酸检测点看起来和以往有些不太一样，\n负责扫码的医护人员面前多了个大纸箱，\n他在里面透过小洞来检测每一个人的二维码', '你在测核酸的时候，注意到小区的绿

In [11]:
#提取方舱文本

from script import Script
import xlsxwriter as xw
import plotly.express as px

data = Script()

mode = 0
quest = 0

ids = [] 
description = []
type1 = []
value = []
succeed = []
fail = []
succeed_type = []
succeed_value = []
fail_type = []
fail_value = []

mode = 0

print (data.arc_quests[0])

for quest in range (0,len(data.arc_quests)):
    ids.append(data.arc_quests[quest]['id'])
    description.append(data.arc_quests[quest]['fields']['description'])
    type1.append(data.arc_quests[quest]['fields']['type'])
    value.append(data.arc_quests[quest]['fields']['value'])
    succeed.append(data.arc_quests[quest]['fields']['succeed'])
    fail.append(data.arc_quests[quest]['fields']['fail'])
    succeed_type.append(str(data.arc_quests[quest]['fields']['succeed_type']))
    succeed_value.append(str(data.arc_quests[quest]['fields']['succeed_value']))
    fail_type.append(str(data.arc_quests[quest]['fields']['fail_type']))
    fail_value.append(str(data.arc_quests[quest]['fields']['fail_value']))

filename = 'arc.xlsx'

workbook = xw.Workbook(filename)
worksheet1 = workbook.add_worksheet("self.arc_quest")#创建子表
worksheet1.activate()#激活表
title = ['序号','任务叙述','判定类型','判定数值','成功检定','失败检定','成功结果','成功数值','失败结果','失败数值',]
worksheet1.write_row('A2',title)

i=2 
for j in range(0,len(description)):
    insertData = ids[j],description[j],type1[j],value[j],succeed[j],fail[j],succeed_type[j],succeed_value[j],fail_type[j],fail_value[j]
    row = 'A'+str(i)
    worksheet1.write_row(row,insertData)
    i+=1
workbook.close()







{'id': 101, 'fields': {'description': '任务简介', 'type': '检定类型wealth,strength,Ingenuity,Cyber', 'value': '检定数值', 'succeed': '检定成功文本', 'fail': '检定失败文本', 'succeed_type': ['default'], 'succeed_value': ['1'], 'fail_type': ['default'], 'fail_value': ['-1']}}


In [10]:
#从excel生成任务
import pandas as pd
import json

#目前到出的list为字符串，需要用筛选剔除掉[]前后的引号引号。

"""
sheet_name用来指定选择哪个子表
header用来指定表头，默认0
默认为None，解析所有列。
如果为str，则表示Excel列字母和列范围的逗号分隔列表（例如“ A：E”或“ A，C，E：F”）。范围全闭。
如果为int，则表示解析到第几列。
如果为int列表，则表示解析那几列。
"""
getdata=pd.read_excel(r'quest_generator.xlsx',sheet_name="self.random_quest",header=0)

#显示前四行数据
#getdata.head()

#显示各项名字，行数 ，每列的数据数量
print(getdata.info())

print(getdata.values[1][0])

random_quests = []


for i in range(1,len(getdata)):
    dict={
    'id':chr,
    "fields":{
        "description":"任务简介",
        "type":"检定类型wealth,strength,Ingenuity,Cyber",
        "value":"检定数值",
        "succeed":"检定成功文本",
        "fail":"检定失败文本",
        "succeed_type":[],
        "succeed_value":[],
        "fail_type":[],
        "fail_value":[],
        }
    }
    dict['id'] = getdata.values[i][0]
    dict['fields']['description'] = getdata.values[i][1]
    dict['fields']['type'] = getdata.values[i][2]
    dict['fields']['value'] = getdata.values[i][3]
    dict['fields']['succeed'] = getdata.values[i][4]
    dict['fields']['fail'] = getdata.values[i][5]
    dict['fields']['succeed_type'] = getdata.values[i][6]
    dict['fields']['succeed_value'] = getdata.values[i][7]
    dict['fields']['fail_type'] = getdata.values[i][8]
    dict['fields']['fail_value'] = getdata.values[i][9]
    random_quests.append(dict)
    
print(random_quests[2])

filename = 'quest_translation.txt'
with open(filename,'w',encoding='utf_8') as f:
    for i in range(len(random_quests)):
        f.write('{\n')
        for k,v in random_quests[i].items():
            f.write(f"\'{str(k)}\':{str(v)},\n")
        f.write('},\n')
    f.write(' \n')
        
f.close()
        


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   序号      47 non-null     int64 
 1   任务叙述    47 non-null     object
 2   判定类型    47 non-null     object
 3   数值      47 non-null     int64 
 4   成功检定    47 non-null     object
 5   失败检定    47 non-null     object
 6   成功结果    47 non-null     object
 7   成功数值    47 non-null     object
 8   失败结果    47 non-null     object
 9   失败数值    47 non-null     object
dtypes: int64(2), object(8)
memory usage: 3.8+ KB
None
2
{'id': 4, 'fields': {'description': '你刷手机的时候，看见沙雕群友们在讨论一篇网文。\n但你', 'type': 'Cyber', 'value': 8, 'succeed': '他们的讨论引起了你的兴趣。\n寻找原文需要一点时间，但这并不能难倒身经百战的你。\n在这篇小说中，一位金丹后期的老祖想要化神。\n他所处之地人气聚集，此前还有人飞升出过大乘，他有此心也并不奇怪。\n可这最后一道雷劫看似平平，却直击人的本源之力。\n此雷劫下众生平等，越是矫揉造作，越是不愿意出真力者越要吃苦。\n你看了一会就关掉了这本小说，你觉得它写得烂透了。', 'fail': '你对网络小说毫无兴趣，继续沉迷水群。', 'succeed_type': "['mood']", 'succeed_value': "['1']", 'fail_type': "['default']", 'fail